<a href="https://colab.research.google.com/github/seobando/TradingBot/blob/main/7_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Colab Set up
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/1 Formal Training/2 Msc. Ciencia de los datos/Semestre IV/Trader

In [ ]:
## Libraries
import torch
import random
import numpy as np
import pandas as pd
from env import environment
import time
from datetime import datetime
from mpl_toolkits import mplot3d
from collections import deque
from ddpg_agent import Agent
import matplotlib.pyplot as plt
env = environment()

In [ ]:
import time
from google.colab import output
import IPython
out = display(IPython.display.Pretty('Starting'), display_id=True)
time.sleep(1)

In [ ]:
## Define Agent
def ddpg_agent(experiment,InitialBudget,state_list,df,random_seed,fc1, fc2, fc3, fc4, fc5, fc6, fc7, fc8, fc9, fc10, fc11,learning_period, update_factor,layer_type,minibatch_size):       

    possibilities = {0:"Hold",1:"Buy",2:"Sell"} 
    set_size = 3
    n_episodes = 1000
    num_agents = 1
    action_size = 3
    state_size = len(state_list)
    print_every = 1
    deque_size = 10
    scores_deque = deque(maxlen=deque_size)
    duration_deque = deque(maxlen=deque_size)
    performance_deque = deque(maxlen=deque_size)
    global_score = []
    size = len(df.index)
    max_t = len(df.index)
    reward = 0
    average_score = 0
    average_performance = 0
    average_duration = 0  

    # Initialize agent
    agent = Agent(state_size, 
                action_size, 
                random_seed,
                fc1, fc2, fc3, fc4, fc5, fc6, fc7, fc8, fc9, fc10, fc11,
                learning_period, update_factor,
                layer_type,
                minibatch_size  
                )
    
    # Iterate over episodes
    for i_episode in range(1, n_episodes+1):

        start_time = time.time()
        done = False
        Budget = InitialBudget
        BuyPrice = 0
        scores = np.zeros(num_agents)
        t = 0
        score = 0
        agent.reset()

        while True:

            price = df["Close"].iloc[t]
            date = df["Date"].iloc[t]
            state = env.row_values(df,t,state_list,num_agents,state_size)
            actions = agent.act(state)
            reward,done,action,BuyPrice,Budget = env.step(done,reward,actions,price,BuyPrice,Budget,InitialBudget)      
            scores += reward

            # Game Pass
            if t+1 >= max_t:
                break    

            next_state = env.row_values(df,t+1,state_list,num_agents,state_size)    
            agent.step(state, actions, reward, next_state, done,t,minibatch_size)  
            t+=1

            # Game Lose
            if done:
                break                          

            # TEST
            performance = round(Budget/InitialBudget-1,6)    
            #print("Action: ", possibilities[action], "Budget: ", Budget,"InitialBudget: ", InitialBudget, "Performance: ", performance)
                
            # TEST
            out.update(IPython.display.Pretty("Experiment: {experiment},Episode: {i_episode}, date:  {date}, action: {action}, score: {score}, steps: {steps}, Initial Budget: {initial_budget} , Budget: {budget}, Performance: {performance}".format(experiment = experiment, i_episode = i_episode, date = date, action = possibilities[action] , score = round(np.mean(scores),2), steps = t, initial_budget = round(InitialBudget,2), budget = round(Budget,2), performance = performance)))
            time.sleep(1)

        score = np.mean(scores)
        scores_deque.append(score)
        average_score = round(np.mean(scores_deque),6)

        performance = round(Budget/InitialBudget-1,2)
        performance_deque.append(performance)
        average_performance = round(np.mean(performance_deque),2)

        end_time = time.time()
        duration = round(end_time - start_time,2)
        duration_deque.append(duration)
        average_duration = round(np.mean(duration_deque),2)
        global_score.append(score)

        # Model valiation
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth') 
    
        #print("Experiment: {experiment} ,Episode: {i_episode}, Avg Performance: {average_performance}, Avg Score: {average_score}, Avg Duration: {average_duration}, Budget: {budget}, Performance: {performance}".format(experiment = experiment, i_episode = i_episode, average_performance = average_performance, average_score = average_score, average_duration = average_duration, budget = round(Budget,2), performance = performance))        

    return global_score

In [ ]:
## Load Data
experiments = pd.read_csv("data/experiment.csv").drop("Unnamed: 0",axis=1)
number_experiments = 1

dataset = "data/data_daily_interpreted_BTC-USD.csv"
data = pd.read_csv(dataset)
df = data[data["Date"]>= "2020-01-01"]

In [ ]:
## Implement Agent
kpis = ["Open_interpreted",
        "High_interpreted",
        "Low_interpreted",
        "MA_short_interpreted",
        "MA_long_interpreted",
        "EMA_short_interpreted",
        "EMA_long_interpreted",
        "MACD_interpreted",
        "BIAS_short_interpreted", 
        "BIAS_long_interpreted", 
        #"VR", 
        "OBV_interpreted"
        ]
    
results_structure = ["Experiment",
                     "Avg Performance",
                     "Avg Time",
                     "Avg Score"
                    ]

results = pd.DataFrame(columns=results_structure)        
# Loop
for i in range(number_experiments):

    ## Experiment
    experiment = experiments["Experiment"].iloc[i]
    ## Start Period

    # End Period

    ## KPIS
    state_list = kpis
    ## Nodes
    nodes = int(experiments["Nodes"].iloc[i]) 
    fc1 = nodes 
    fc2 = nodes
    fc3 = nodes
    fc4 = nodes
    fc5 = nodes
    fc6 = nodes
    fc7 = nodes 
    fc8 = nodes
    fc9 = nodes
    fc10 = nodes
    fc11 = nodes
    ## Layers Type
    layer_amount_type = experiments["Layer Amount Type"].iloc[i]
    ## Layer Type Critic
    layer_type = experiments["Layers Type"].iloc[i]
                
    if layer_type != "Constant":
        if nodes == int(600):
          fc6 = 600
          fc7 = 500
          fc8 = 400
          fc9 = 300
          fc10 = 300
          fc11 = 300
        else:
          fc6 = 1024
          fc7 = 512
          fc8 = 256
          fc9 = 128
          fc10 = 128
          fc11 = 128              
    ## Learning Period
    learning_period = experiments["Learning Period"].iloc[i]
    ## Update Factor 
    update_factor = experiments["Update Factor"].iloc[i]
    ## Random Seed
    random_seed = 42
    ## Minibatch Size
    minibatch_size = experiments["Minibatch"].iloc[i]         
    ## Initial Budget
    InitialBudget = df["High"].max()*10

    #TEST  
    test = [
            experiment,
            nodes ,
            fc1 ,
            fc2,
            fc3,
            fc4,
            fc5,
            fc6,
            fc7,
            fc8,
            fc9,
            fc10,
            fc11,
            layer_amount_type,
            layer_type,
            learning_period,
            update_factor,
            random_seed,
            minibatch_size,    
            InitialBudget
          ]

    #print(test)      
    #print("Initial Budget: ", InitialBudget)

    global_score = ddpg_agent(experiment,InitialBudget,state_list,df,random_seed,fc1, fc2, fc3, fc4, fc5, fc6, fc7, fc8, fc9, fc10, fc11,learning_period, update_factor,layer_amount_type,minibatch_size)

In [ ]:
## Visualization
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1,len(global_score)+1), global_score)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()